<a href="https://colab.research.google.com/github/LEhongPhuc2/nh-m-4/blob/main/Hermes_3_Llama_3_1_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

In [ ]:
!pip install -U lightrag[ollama]

In [ ]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient
from IPython.display import display, Markdown
import ipywidgets as widgets

# Define the template for the chatbot
qa_template = r"""<SYS>
You are a helpful assistant.
</SYS>
:User   {{input_str}}
You:"""

# Create a SimpleQA class for the chatbot
class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

# Initialize the chatbot model
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)

In [ ]:
# Create input and output widgets
input_box = widgets.Text(
    description='Bạn:',
    placeholder='Nhập câu hỏi của bạn ở đây...'
)

output_area = widgets.Output()

def on_submit(sender):
    user_input = input_box.value
    with output_area:
        output_area.clear_output()
        response = qa(user_input)
        display(Markdown(f"**Answer:** {response.data}"))
    input_box.value = ''  # Clear the input box after submission

input_box.on_submit(on_submit)

# Display the interface
display(input_box, output_area)

Text(value='', description='Bạn:', placeholder='Nhập câu hỏi của bạn ở đây...')

Output()